IBM Ponder This Challenge April 2023 - python docplex version

In [28]:
n=24
r=range(0,n)

str="\
000001000000000001110011\
110100010110101000010011\
011101110000001101001110\
000110111000110101101100\
101101011010010011101010\
111000100101110100101000\
110001011100000000000101\
100000010001100000000010\
000110010010110110101001\
011101101011111011100000\
011000101010111011111100\
100011110010000100100111\
000111010010100010001110\
011001010001001111110101\
110001000010111000100000\
000000101100101000101001\
111001010010010011110110\
100000110001111111011010\
110100000011100100110010\
101000110111001110010000\
110000000010011100100101\
111111011011111100010101\
000000000110101011100000\
110001111100000011001111"

assert len(str)==n*n



Turn the string into a 2D array

In [29]:
ar=[[int(str[j*n+i]) for i in r] for j in r]

In [30]:
print(ar)

[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1], [1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1], [0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0], [0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0], [1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1], [0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0], [1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1], [0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0], [0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 

In [31]:
from docplex.mp.model import Model 

mdl = Model(name='April2023ponderthis')

First let us forget about the  constraint that a lightbulb has to be off to be chosen.

In [32]:
!  Do we switch this lightbulb ?
x = mdl.binary_var_matrix(keys1=r,keys2=r,name="x")

! dvar int v[r][r]; // Integer decision variable to check that the light is on finally
v = mdl.integer_var_matrix(keys1=r,keys2=r,name="v")

for i in r:
    for j in r:
        mdl.add(ar[i][j]+x[i,j]+mdl.sum(x[k,j] for k in r if k!=i)+mdl.sum(x[i,k] for k in r if k!=j)==v[i,j]*2+1)

mdl.solve()

/usr/bin/sh: Do: command not found
/usr/bin/sh: dvar: command not found
/usr/bin/sh: //: Is a directory


docplex.mp.solution.SolveSolution(obj=0,values={x_0_0:1,x_0_6:1,x_0_8:1,..

Now let's use some python to handle the constraint that a lightbulb has to be off to be chosen.
For that let's find another one if necessary to turn on and then off

In [33]:
xyok=[(i,j) for i in r for j in r if x[i,j].solution_value==1]

def apply(ar,x,y):
    #The call had to be off
    assert 0==ar[x][y]
    print("(",y+1,",",n-x,")",",",end="")
    ar[x][y]=1-ar[x][y]
    for k in r:
        if k!=x:
            ar[k][y]=1-ar[k][y]
    for k in r:
        if k!=y:
            ar[x][k]=1-ar[x][k]
            
            
def lookforoption(ar,x,y):
    for k in r:
        if k!=y:
            if (ar[x][k]==0):
                return [x,k]
    for k in r:
        if k!=x:
            if (ar[k][y]==0):
                return [k,y]
    


In [34]:
print("[",end="")
for index,k in enumerate(xyok):
    ki=k[0]
    kj=k[1]
    switchcell=ar[ki][kj]
    option=[-1,-1]
    if switchcell==1:
        option=lookforoption(ar,ki,kj)
        apply(ar,option[0],option[1])
    apply(ar,ki,kj)
    if switchcell==1:
        apply(ar,option[0],option[1])
    if (index+1) % 5==0:
        print()
print("]")          

[( 1 , 24 ) ,( 6 , 24 ) ,( 7 , 24 ) ,( 6 , 24 ) ,( 9 , 24 ) ,( 6 , 24 ) ,( 11 , 24 ) ,( 6 , 24 ) ,( 12 , 24 ) ,
( 6 , 24 ) ,( 13 , 24 ) ,( 6 , 24 ) ,( 15 , 24 ) ,( 18 , 24 ) ,( 1 , 24 ) ,( 19 , 24 ) ,( 1 , 24 ) ,( 20 , 24 ) ,
( 21 , 24 ) ,( 23 , 24 ) ,( 1 , 23 ) ,( 2 , 23 ) ,( 3 , 23 ) ,( 2 , 23 ) ,( 5 , 23 ) ,
( 2 , 23 ) ,( 11 , 23 ) ,( 2 , 23 ) ,( 13 , 23 ) ,( 2 , 23 ) ,( 14 , 23 ) ,( 2 , 23 ) ,( 15 , 23 ) ,( 2 , 23 ) ,( 16 , 23 ) ,( 2 , 23 ) ,
( 17 , 23 ) ,( 18 , 23 ) ,( 1 , 23 ) ,( 19 , 23 ) ,( 1 , 23 ) ,( 2 , 23 ) ,( 22 , 23 ) ,( 2 , 23 ) ,( 1 , 23 ) ,( 24 , 23 ) ,( 1 , 23 ) ,
( 1 , 22 ) ,( 2 , 22 ) ,( 3 , 22 ) ,( 4 , 22 ) ,( 1 , 22 ) ,( 8 , 22 ) ,( 1 , 22 ) ,
( 9 , 22 ) ,( 11 , 22 ) ,( 12 , 22 ) ,( 13 , 22 ) ,( 2 , 22 ) ,( 16 , 22 ) ,( 2 , 22 ) ,
( 1 , 22 ) ,( 18 , 22 ) ,( 1 , 22 ) ,( 2 , 22 ) ,( 22 , 22 ) ,( 2 , 22 ) ,( 23 , 22 ) ,( 24 , 22 ) ,( 3 , 21 ) ,( 1 , 21 ) ,( 3 , 21 ) ,
( 1 , 21 ) ,( 4 , 21 ) ,( 1 , 21 ) ,( 5 , 21 ) ,( 1 , 21 ) ,( 6 , 21 ) ,( 1 , 21 ) ,( 8 , 21 ) ,( 11

In [35]:
for i in r:
    for j in r:
        assert ar[i][j]==1